In [1]:
# Imports
from subsbml import System, createSubsystem, combineSystems, createNewSubsystem, createBasicSubsystem, SimpleModel, SimpleReaction
import numpy as np
import pylab as plt

from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import libsbml
import bokeh.io
import bokeh.plotting

## Cells B and P communicating with each other ##
Cell_B = Bistable SWitch

Cell_P = Produce2N

External environment = rR1 reservoir

In [2]:
ss1 = createSubsystem ("SBML_Models/BiSwitch_CRN.xml")
ss2 = createSubsystem ("SBML_Models/rR1_external_reservoir.xml")

# Membrane 
mb1 = createSubsystem("SBML_Models/rR1_membrane1_detailed.xml", membrane = True)
### Cell B ###
cell_B = System("cell_B")

cell_B.setInternal([ss1])
cell_B.setExternal([ss2])
cell_B.setMembrane(mb1)

cell_B_model = cell_B.getModel()
cell_B_model.writeSBML('SBML_Models/cell_B_model.xml')
print (cell_B)

The subsystem from SBML_Models/rR1_membrane1_detailed.xml has multiple compartments


In [3]:
ss3 = createSubsystem("SBML_Models/Full_Produce.xml")

# Membrane 
mb1 = createSubsystem("SBML_Models/rR1_membrane1_detailed.xml", membrane = True)

### Cell P ###
cell_P = System("cell_P")

cell_P.setInternal([ss3])
cell_P.setExternal([ss2])
cell_P.setMembrane(mb1)

cell_P_model = cell_P.getModel()
cell_P_model.writeSBML('SBML_Models/cell_P_model.xml')

The subsystem from SBML_Models/rR1_membrane1_detailed.xml has multiple compartments


1

In [9]:
#### Combine both Systems ####
combined_model = combineSystems([cell_B, cell_P])

combined_model.shareSubsystems([ss2], ListOfSharedResources=['rna_rR1_e'])
#combined_model.ListOfSharedResources = ['rna_rR1_e']
combined_model.changeLog

combined_model.writeSBML('combined.xml')
#### Shared external subsystem ###

1

In [10]:
# Cell_B Identifiers #

B_id1 = combined_model.getSpeciesByName('complex_Core1_ON').getId()
B_id2 = cell_B_model.getSpeciesByName('Core1_OFF').getId()
B_id3 = cell_B_model.getSpeciesByName('complex_Core2_ON').getId()
B_id4 = cell_B_model.getSpeciesByName('Core2_OFF').getId()
B_id5 = cell_B_model.getSpeciesByName("rna_rR2").getId()
B_id6 = cell_B_model.getSpeciesByName("rna_rR1", compartment = 'cell_B_internal').getId()
B_id7 = cell_B_model.getSpeciesByName("rna_rR1_e", compartment = 'cell_B_external').getId()
B_id8 = cell_B_model.getSpeciesByName("complex_Core2_AI").getId()
B_id9 = cell_B_model.getSpeciesByName("complex_Core1_AI").getId()
B_id10 = cell_B_model.getSpeciesByName("dna_dA2").getId()
B_id11 = cell_B_model.getSpeciesByName("dna_dA1").getId()

ValueError: The species complex_Core1_ON not found.

In [ ]:
# Cell_P Identifiers #

P_id1 = cell_P_model.getSpeciesByName('complex_Prod2_ON').getId()
P_id2 = cell_P_model.getSpeciesByName('Prod2_OFF').getId()
P_id3 = cell_P_model.getSpeciesByName('DFHBI').getId()
P_id4 = cell_P_model.getSpeciesByName('BrocApt').getId()
P_id5 = cell_P_model.getSpeciesByName("complex_BrocApt_DFHBI").getId()
P_id6 = cell_P_model.getSpeciesByName("rna_rR1", compartment = 'cell_P_internal').getId()
P_id7 = cell_P_model.getSpeciesByName("rna_rR1_e", compartment = 'cell_P_external').getId()

In [ ]:
# Simulate with Bioscrape #
timepoints = np.linspace(0,28800,1000)
B,_ = combined_model.simulateWithBioscrape(timepoints)
P,_ = cell_P_model.simulateWithBioscrape(timepoints)
timepoints = timepoints/3600

# For label convenience
x = 'Time (hours)'
y = 'Concentration (uM)'

bokeh.io.output_notebook()
a = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
a.circle(timepoints, B[B_id1], legend_label = "Core1 ON" , color = "blue")
a.circle(timepoints, B[B_id3], legend_label = "Core2 ON", color = "red")
a.legend.click_policy="hide"
a.legend.location="bottom_right"

b = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
b.circle(timepoints, P[P_id1], legend_label = "Prod2 ON" , color = "blue")
b.circle(timepoints, P[P_id2], legend_label = "Prod2 OFF", color = "red")
b.legend.click_policy="hide"
b.legend.location="bottom_right"

bokeh.io.show(row(a, b))
warnings.filterwarnings("ignore")

In [ ]:
a = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
a.circle(timepoints, B[B_id6], legend_label = "rR1 internal" , color = "blue")
a.legend.click_policy="hide"
a.legend.location="bottom_right"

b = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
b.circle(timepoints, P[P_id6], legend_label = "rR1 Prod i" , color = "blue")
b.circle(timepoints, P[P_id7], legend_label = "rR1 Prod e", color = "red")
b.legend.click_policy="hide"
b.legend.location="bottom_right"

c = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
c.circle(timepoints, B[B_id7], legend_label = "rR1 external", color = "red")
c.legend.click_policy="hide"
c.legend.location="bottom_right"

bokeh.io.show(row(a, c, b))
warnings.filterwarnings("ignore")